In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#Define path
path = r'C:\Users\avamk\03-2023 Instacart Basket Analysis'

In [3]:
#Import customer data set as a dataframe
df_customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
#Change column names so they are all consistent with lowercase letters, fix misspellings, and give more context to those that need it.
df_customer.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [5]:
df_customer.rename(columns = {'Surnam' : 'surname'}, inplace = True)

In [6]:
df_customer.rename(columns = {'Gender' : 'gender'}, inplace = True)

In [7]:
df_customer.rename(columns = {'STATE' : 'state'}, inplace = True)

In [8]:
df_customer.rename(columns = {'Age' : 'age'}, inplace = True)

In [9]:
df_customer.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)

In [10]:
df_customer.rename(columns = {'fam_status' : 'family_status'}, inplace = True)

In [11]:
#Check the renamed columns
df_customer.head()

user_id first_name   surname  gender       state  age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   number_of_dependants family_status  income  
0                     3       married  165665  
1                     0        single   59285  
2                     2       married   99568  
3                     0        single   42049  
4                     1       married   40374

In [12]:
#Check the type of the user_id column, since this will be our common variable when combining data sets
df_customer['user_id'].dtype

dtype('int64')

In [13]:
#Change user_id column into a string
df_customer['user_id'] = df_customer['user_id'].astype('str')

In [14]:
#Check for mixed-type data
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

first_name


It looks like the first_name column contains mixed-type data.

In [15]:
#Check for missing values
df_customer.isnull().sum()

user_id                     0
first_name              11259
surname                     0
gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependants        0
family_status               0
income                      0
dtype: int64

It shows that the first_name column contains 11259 missing values.

In [17]:
#Create list of the missing values
df_nan = df_customer[df_customer['first_name'].isnull() == True]

In [18]:
df_nan

user_id first_name   surname  gender         state  age date_joined  \
53       76659        NaN   Gilbert    Male      Colorado   26    1/1/2017   
73       13738        NaN     Frost  Female     Louisiana   39    1/1/2017   
82       89996        NaN    Dawson  Female        Oregon   52    1/1/2017   
99       96166        NaN   Oconnor    Male      Oklahoma   51    1/1/2017   
105      29778        NaN    Dawson  Female          Utah   63    1/1/2017   
...        ...        ...       ...     ...           ...  ...         ...   
206038  121317        NaN    Melton    Male  Pennsylvania   28   3/31/2020   
206044  200799        NaN  Copeland  Female        Hawaii   52    4/1/2020   
206090  167394        NaN     Frost  Female        Hawaii   61    4/1/2020   
206162  187532        NaN     Floyd  Female    California   39    4/1/2020   
206171  116898        NaN   Delgado  Female      Colorado   23    4/1/2020   

        number_of_dependants family_status  income  
53                         2       married   41709  
73                         0        single   82518  
82                         3       married  117099  
99                         1       married  155673  
105                        3       married  151819  
...                      ...           ...     ...  
206038                     3       married   87783  
206044                     2       married  108488  
206090                     1       married   45275  
206162                     0        single   56325  
206171                     2       married   59222  

[11259 rows x 10 columns]

Although there are quite a few missing values, I am going to leave it for now since I don't know if the first_name column is important yet. I would assume it wouldn't affect our analysis at all, but I would like to be sure before deleting.

In [19]:
#Check for duplicates
df_dups = df_customer[df_customer.duplicated()]

In [20]:
df_dups

Empty DataFrame
Columns: [user_id, first_name, surname, gender, state, age, date_joined, number_of_dependants, family_status, income]
Index: []

No duplicates were found.

In [21]:
#Import our prepared instacart data
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_agg.pkl'))

In [23]:
df_ords_prods['user_id'] = df_ords_prods['user_id'].astype('str')

In [25]:
#Merge datasets 
df_merged = df_ords_prods.merge(df_customer, on = 'user_id')

In [26]:
df_merged.head()

order_id user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329       1    prior             1          2                  8   
1   2398795       1    prior             2          3                  7   
2    473747       1    prior             3          3                 12   
3   2254736       1    prior             4          4                  7   
4    431534       1    prior             5          4                 15   

   days_since_prior_order  product_id  add_to_cart_order  reordered  ...  \
0                     NaN         196                  1          0  ...   
1                    15.0         196                  1          1  ...   
2                    21.0         196                  1          1  ...   
3                    29.0         196                  1          1  ...   
4                    28.0         196                  1          1  ...   

         order_freq_flag  first_name  surname  gender    state age  \
0  Non-frequent customer       Linda   Nguyen  Female  Alabama  31   
1  Non-frequent customer       Linda   Nguyen  Female  Alabama  31   
2  Non-frequent customer       Linda   Nguyen  Female  Alabama  31   
3  Non-frequent customer       Linda   Nguyen  Female  Alabama  31   
4  Non-frequent customer       Linda   Nguyen  Female  Alabama  31   

  date_joined number_of_dependants family_status  income  
0   2/17/2019                    3       married   40423  
1   2/17/2019                    3       married   40423  
2   2/17/2019                    3       married   40423  
3   2/17/2019                    3       married   40423  
4   2/17/2019                    3       married   40423  

[5 rows x 34 columns]

In [27]:
#Export merged data as a pickle file
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_produducts_customers.pkl'))